In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable

from torchvision import transforms
from torchvision import datasets
print(torch.cuda.is_available())
torch.backends.cudnn.benchmark = True
is_cuda = True

True


In [2]:
train_dataset = datasets.MNIST(root='./data',
                           train=True,
                           transform=transforms.ToTensor(),
                           download=True)

test_dataset = datasets.MNIST(root = './data',
                              train = False,
                              transform=transforms.ToTensor(),
                              download = True)

In [3]:
batch_size = 100
n_iters = 6000
num_epochs = int(n_iters / (len(train_dataset) / batch_size))

In [4]:
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset,batch_size, shuffle=False)

Here you have to calculate in_features. (i.e. no. of features for a single number photo)
P : padding
O : output height/lenght
I : input height
K : kernel size
S : Stride

P = (k-1)/2 for O == I

O = (I- K+ 2P)/S  +  1  

here, after 
cnn1 : (28,28) > (16,28,28)  16 is no. of kernel used
maxpool1 : (16,28,28) > (16, 14, 14) as filtersize used in maxpool = 2
cnn2 : (16, 14, 14) > (32, 14, 14)
maxpool2 : (32,14,14) > (32,7,7)

total : 32*7*7

In [5]:
# class CNNModel(nn.Module):
#     def __init__(self):
#         super(CNNModel, self).__init__()
        
#         # Convolution 1
#         self.cnn1 = nn.Conv2d(in_channels=1, out_channels=20, kernel_size=5, stride=1, padding=2)
#         self.relu1 = nn.ELU()
        
#         # Max pool 1
#         self.maxpool1 = nn.MaxPool2d(kernel_size=2)
        
#         # Convolution 2
#         self.cnn2 = nn.Conv2d(in_channels=20, out_channels=40, kernel_size=5, stride=1, padding=2)
#         self.relu2 = nn.ELU()

#         # Max pool 2
#         self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        
#         # Fully connected (readout)
#         # The trick to calc in_features is put anything there, 
#         # while output = model(image), the error will tell you the right dimention of in_features
#         self.fc = nn.Linear(in_features=40*7*7, out_features=10)
        
#     def forward(self, x):
#         out = self.cnn1(x)
#         out = self.relu1(out)
#         out = self.maxpool1(out)
#         out = self.cnn2(out)
#         out = self.relu2(out)
#         out = self.maxpool2(out)
        
#         # Resize
#         # Original size : (100, 32, 7, 7)
#         # out.size(0) : 100
#         # New out size : (100, 32*7*7)
#         out = out.view(out.size(0), -1)
        
#         # Linear function (readout)
#         out = self.fc(out)
        
#         return out

In [6]:
# model = CNNModel()
# if is_cuda == True:
#     model.cuda()

40*7*7

1960

In [7]:
model = nn.Sequential(
    nn.Conv2d(in_channels=1, out_channels=20, kernel_size=5, stride=1, padding=2),
    nn.ELU(),
    
    nn.MaxPool2d(kernel_size=2),
    
    nn.Conv2d(in_channels=20, out_channels=40, kernel_size=5, stride=1, padding=2),
    nn.ELU(),
    
    nn.MaxPool2d(kernel_size=2),
    
    nn.Flatten(),
    nn.Linear(in_features=40*7*7, out_features=10)
)

if is_cuda == True:
    model.cuda()

In [8]:
model

Sequential(
  (0): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (1): ELU(alpha=1.0)
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(20, 40, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (4): ELU(alpha=1.0)
  (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Flatten()
  (7): Linear(in_features=1960, out_features=10, bias=True)
)

In [9]:
criterion = nn.CrossEntropyLoss()

In [10]:
learning_rate = 0.01

optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

In [11]:
# No. of parameters layer
print(len(list(model.parameters())))

# No of parameters in 1st layer:
print(list(model.parameters())[0].size())

# No of parameters in 2nd layer:
print(list(model.parameters())[1].size())

# Conv. 2: 32 kernels with depth 16 each:
print(list(model.parameters())[2].size())

# Conv. 2 Bias : 32 kernels with depth 16 each:
print(list(model.parameters())[3].size())

# Fully connected layer:
print(list(model.parameters())[4].size())

# Fully connected layer size:
print(list(model.parameters())[5].size())

6
torch.Size([20, 1, 5, 5])
torch.Size([20])
torch.Size([40, 20, 5, 5])
torch.Size([40])
torch.Size([10, 1960])
torch.Size([10])


In [12]:
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images)
        labels = Variable(labels)
        if is_cuda == True:
            images = images.cuda()
            labels = labels.cuda()
        
        optimizer.zero_grad()
        
        outputs = model(images)
        
        loss = criterion(outputs, labels)
        
        loss.backward()
        
        optimizer.step()
        
        iter +=1
        if iter % 500 == 0:
            correct = 0
            total = 0
            
            for images_t, labels_t in test_loader:
                images_t = Variable(images_t)
                labels_t = Variable(labels_t)
                if is_cuda == True:
                    images_t = images_t.cuda()
                    labels_t = labels_t.cuda()
                    
                outputs_t = model(images_t)
                _, predicted_t = torch.max(outputs_t.detach(), 1)
                total += labels_t.size()[0]
                correct += (predicted_t == labels_t).sum()
                
            accuracy = 100 * int(correct) / total
            
            print("Iteration: {}, loss: {}, Accuracy: {}".format(iter, loss, accuracy))

Iteration: 500, loss: 0.3400763273239136, Accuracy: 90.13
Iteration: 1000, loss: 0.23290015757083893, Accuracy: 93.46
Iteration: 1500, loss: 0.26063305139541626, Accuracy: 94.67
Iteration: 2000, loss: 0.21753603219985962, Accuracy: 95.69
Iteration: 2500, loss: 0.1353742927312851, Accuracy: 96.37
Iteration: 3000, loss: 0.1647464632987976, Accuracy: 96.73
Iteration: 3500, loss: 0.04651112109422684, Accuracy: 97.29
Iteration: 4000, loss: 0.04212779551744461, Accuracy: 97.46
Iteration: 4500, loss: 0.04117724299430847, Accuracy: 97.78
Iteration: 5000, loss: 0.04253332316875458, Accuracy: 97.93
Iteration: 5500, loss: 0.038836121559143066, Accuracy: 98.06
Iteration: 6000, loss: 0.05933374911546707, Accuracy: 98.24


In [13]:
predicted_t

tensor([8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 0, 1, 2, 3, 4, 7, 8, 9, 7, 8, 6, 4, 1,
        9, 3, 8, 4, 4, 7, 0, 1, 9, 2, 8, 7, 8, 2, 6, 0, 6, 5, 3, 3, 3, 9, 1, 4,
        0, 6, 1, 0, 0, 6, 2, 1, 1, 7, 7, 8, 4, 6, 0, 7, 0, 3, 6, 8, 7, 1, 5, 2,
        4, 9, 4, 3, 6, 4, 1, 7, 2, 6, 6, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2,
        3, 4, 5, 6], device='cuda:0')

In [14]:
labels_t

tensor([8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 0, 1, 2, 3, 4, 7, 8, 9, 7, 8, 6, 4, 1,
        9, 3, 8, 4, 4, 7, 0, 1, 9, 2, 8, 7, 8, 2, 6, 0, 6, 5, 3, 3, 3, 9, 1, 4,
        0, 6, 1, 0, 0, 6, 2, 1, 1, 7, 7, 8, 4, 6, 0, 7, 0, 3, 6, 8, 7, 1, 5, 2,
        4, 9, 4, 3, 6, 4, 1, 7, 2, 6, 5, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2,
        3, 4, 5, 6], device='cuda:0')

In [15]:
(predicted_t == labels_t)

tensor([ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True, False,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True],
       device='cuda:0')